In [ ]:
from itertools import combinations
from collections import defaultdict, namedtuple, deque
from math import isclose
from operator import eq

import colour
import uuid

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.patches import Polygon
from matplotlib.collections import PolyCollection

def determine_base_K(p, c, s, o):
    # If it's on a line, lower it to the origin
    if isclose(((c * p[0] + s * p[1]) % 1), o, rel_tol=1e-5, abs_tol=1e-05):
        p_distance = np.sqrt(p[0]**2 + p[1]**2)
        p_normal_origin = [-p_ / p_distance * .5 for p_ in p]
        p = [p + pno for p, pno in zip(p, p_normal_origin)]

    base_k = np.ceil(c * p[0] + s * p[1] - o)
    return base_k

def determine_K(c, x, s, y, o):
    return np.ceil(c * x + s * y - o)

class Point:
    def __init__(self, k_values, cos_list, sin_list, mid_point):
        self.k_values = k_values
        
        self.location = np.array([0, 0], dtype=float)
        for k, c, s in zip(k_values, cos_list, sin_list):
            self.location += k * np.array([c, s])
        self.location -= mid_point

class Line:
    # a*x + b*y = c
    def __init__(self, a, b, o, k, grid):
        self.a = a
        self.b = b
        self.c = o + k
        self.offset = o
        self.k = int(k)
        self.grid = grid

        self.first_angle = np.arctan2(-a, b)
        if self.first_angle < 0:
            self.second_angle = self.first_angle + np.pi
        else:
            self.second_angle = self.first_angle - np.pi

        # crossing counter clockwise, first angle is k + 1, second is k
        # because of how np.arctan2 works?
        Angle = namedtuple('Angle', ['value', 'k', 'grid'])

        self.angles = [Angle(self.first_angle, self.k + 1, grid), 
                       Angle(self.second_angle, self.k, grid)]

        # Find closes point to the origin
        self.origin_x = (self.a * self.c) / (self.a**2 + self.b**2)
        self.origin_y = (self.b * self.c) / (self.a**2 + self.b**2)

        self.perpendicular_angle = np.arctan2(b, a)

        # Keep this stuff in here for testing, but add parameter to exclude it later
        self.p1 = [np.cos(self.perpendicular_angle) * .25 + self.origin_x,
              np.sin(self.perpendicular_angle) * .25 + self.origin_y ]
        self.p2 = [np.cos(self.perpendicular_angle) * -.25 + self.origin_x,
              np.sin(self.perpendicular_angle) * -.25 + self.origin_y ]

        self.K = [self.determine_K(self.p1), self.determine_K(self.p2)]

    def __str__(self):
        return f'a={self.a}, b={self.b}, offset={self.offset}, k={self.k}, grid={self.grid}'

    def determine_K(self, p):
        return np.ceil(self.a * p[0] + self.b * p[1] - self.offset)

    # Build function to get lines that hit boundaries
    # Maybe use build intersection function and use a line for each boundary!!!???
    def determine_intersection(self, other_line):

        denom = (self.a * other_line.b) - (other_line.a * self.b)
        if isclose(denom, 0, rel_tol=1e-5, abs_tol=1e-05):
            # Determine if it's the same line

            if isclose(other_line.a, 0, rel_tol=1e-5, abs_tol=1e-05):
                if not isclose(self.a, 0, rel_tol=1e-5, abs_tol=1e-05):
                    # Different lines
                    return None
                a_proportion = 0
            else:
                a_proportion = self.a / other_line.a
            
            if isclose(other_line.b, 0, rel_tol=1e-5, abs_tol=1e-05):
                if not isclose(self.b, 0, rel_tol=1e-5, abs_tol=1e-05):
                    # Different lines
                    return None
                b_proportion = 0
            else:
                b_proportion = self.b / other_line.b

            if a_proportion and b_proportion:
                if not isclose(a_proportion, b_proportion):
                    return None

            if isclose(other_line.c, 0, rel_tol=1e-5, abs_tol=1e-05):
                if not isclose(self.c, 0, rel_tol=1e-5, abs_tol=1e-05):
                    # Different lines
                    return None
                c_proportion = 0
            else:
                c_proportion = self.c / other_line.c

            if a_proportion and c_proportion:
                if not isclose(a_proportion, c_proportion, rel_tol=1e-5, abs_tol=1e-05):
                    return None
            
            if b_proportion and c_proportion:
                if not isclose(b_proportion, c_proportion, rel_tol=1e-5, abs_tol=1e-05):
                    return None

            raise OverlappingLines(self, other_line)
        else:
            x = (other_line.b * self.c) - (self.b * other_line.c)
            x = x / denom

            y = (self.a * other_line.c) - (other_line.a * self.c)
            y = y / denom

            return (np.round(x, 7), np.round(y, 7))
        
class OverlappingLines(Exception):
    def __init__(self, line_1, line_2, msg="These two lines are the same line."):
        self.line_1 = line_1
        self.line_2 = line_2
        self.msg = msg
        super().__init__(self.msg)

    def __str__(self):
        return f'{self.msg}\n\t{self.line_1}\n\t{self.line_2}'
    
class Edge:
    '''Class used for tile edges, doesn't relate to the Line class'''
    def __init__(self, point_ids, tile_id):
        self.points = point_ids
        self.tile_ids = [tile_id]

    def add_tile_id(self, tile_id):
        self.tile_ids.append(tile_id)
        
        if len(self.tile_ids) > 2:
            raise OverusedEdge(self)
        
    def __str__(self):
        msg = f'point_ids: {self.points}'
        for t in self.tile_ids:
            msg = msg + f'\n\t{t}'
        return msg
    
class OverusedEdge(Exception):
    '''Happens when the floating point fails on a point with multiple lines'''
    def __init__(self, edge, msg="This edge has more than two tiles."):
        self.edge = edge
        self.msg = msg
        super().__init__(self.msg)

    def __str__(self):
        return f'{self.msg}\n\t{self.edge}'
    
class Tile:
    def __init__(self, tile_points,):
        self.tile_points = tile_points
        self.tile_group = 0

        self.tile_id = uuid.uuid4()
        self.connected = False

        # Don't need to compare distances between angles, always 1 by construction
        self.angles = []
        for i, tile_point in enumerate(self.tile_points):
            previous_point = self.tile_points[(i - 1) % len(self.tile_points)]
            next_point = self.tile_points[(i + 1) % len(self.tile_points)]

            point_location = tile_point.location
            previous_point_location = previous_point.location
            next_point_location = next_point.location

            in_vector = point_location - previous_point_location
            out_vector = next_point_location - point_location
            angle = np.arctan2(in_vector[0]*out_vector[1]-in_vector[1]*out_vector[0],
                            in_vector[0]*out_vector[0]+in_vector[1]*out_vector[1])

            self.angles.append(angle)

    def compare_angles(self, other_polygon):
        if len(self.angles) != len(other_polygon.angles):
            return False

        angles_repeated = self.angles + self.angles
        for a in np.arange(len(self.angles)):
            match = True
            for b in np.arange(len(other_polygon.angles)):
                if not isclose(angles_repeated[a + b], other_polygon.angles[b], rel_tol=1e-5, abs_tol=1e-05):
                    match = False
                    break
            if match:
                return True
        return False
    
class MultiGrid:
    def __init__(self, seed=None, grid_count=None, 
                      grid_bounds=[], rotation=None, 
                      base_point = (),
                      offsets=[], colors=[]):
        self.seed = seed
        self.rng = np.random.default_rng(self.seed)
        
        if not grid_count:
            grid_count = self.rng.integers(low=3, high=20)
        self.grid_count = grid_count

        if not grid_bounds:
            grid_bounds = self.rng.integers(low=2, high=10)
            grid_bounds = [-grid_bounds, grid_bounds+1]
        self.grid_bounds = grid_bounds

        if not rotation:
            rotation = self.rng.uniform(low=0, high=(2 * np.pi / grid_count))
        self.rotation = rotation

        self.cos_list = [np.cos(x * 2 * np.pi / self.grid_count + self.rotation) for x in np.arange(self.grid_count)]
        self.sin_list = [np.sin(x * 2 * np.pi / self.grid_count + self.rotation) for x in np.arange(self.grid_count)]

        if not offsets:
            offsets = self.rng.uniform(size=grid_count)
            offsets = offsets / sum(offsets)
        self.offsets = offsets

        if colors:
            self.colors = colors
        else:
            self.colors = []

        if not base_point:
            r = 25 * np.sqrt(self.rng.random())
            theta = self.rng.random() * 2 * np.pi
            self.base_point = np.array([r * np.cos(theta),
                                        r * np.sin(theta), 0], dtype=float)

        base_k_values = []
        for c, s, o in zip(self.cos_list, self.sin_list, offsets):
            base_k_values.append(determine_base_K(self.base_point, c, s, o))        

        self.grids = []
        for i, (s, c, o, bk) in enumerate(zip(self.sin_list, self.cos_list, offsets, base_k_values)):
            for k in np.arange(*grid_bounds):
                self.grids.append(Line(c, s, o, bk + k, i))

        self.mid_point = np.array([0, 0], dtype=float)
        for k, c, s in zip(base_k_values, self.cos_list, self.sin_list):
            self.mid_point += k * np.array([c, s])

    def update_edges(self, points, tile_id):
        if points in self.edges:
            self.edges[points].add_tile_id(tile_id)
        else:
            self.edges[points] =  Edge(points, tile_id)

    def create_tiles(self):

        self.points = {}

        self.edges = {}  

        self.intersections = defaultdict(set)
        for pair in combinations(self.grids, 2):
            l1 = pair[0]
            l2 = pair[1]
            
            if l1.grid == l2.grid:
                continue

            try:
                intersection = l1.determine_intersection(l2)
                if intersection:
                    self.intersections[intersection].update([l1, l2])
            except OverlappingLines as e: 
                print(e)

        self.base_tiles = []
        self.tiles = {}

        for intersection_points, intersection_lines in self.intersections.items():

            # Maybe, get all the angles in order
            angles = []
            for l in intersection_lines:
                angles.extend(l.angles.copy())
            angles.sort()

            # #start with right above (or below) the first one, (maybe need to get to second+ line to get all the K's) then continue looping through
            # Set up K values for the other lines
            ks = [determine_K(c, intersection_points[0], s, intersection_points[1], o) for c, s, o in zip(self.cos_list, self.sin_list, self.offsets)]

            # Now spin around to ensure the ks are correct on the lines that intersect
            for angle in angles:
                ks[angle.grid] = angle.k

            tile_points = []
            for angle in angles:
                ks[angle.grid] = angle.k

                k_values = tuple(ks)
                if not k_values in self.points:
                    self.points[k_values] = Point(k_values, self.cos_list, self.sin_list, self.mid_point)

                tile_points.append(self.points[k_values])

            t = Tile(tile_points)

            for t_current, t_next in zip(t.tile_points, t.tile_points[1:] + [t.tile_points[0]]):
                # Check left to right, then down to up
                if t_current.location[1] < t_next.location[1]:
                    self.update_edges(points=(t_current.k_values, t_next.k_values), tile_id=t.tile_id)
                elif t_next.location[1] < t_current.location[1]:
                    self.update_edges(points=(t_next.k_values, t_current.k_values), tile_id=t.tile_id)
                elif t_current.location[0] < t_next.location[0]:
                    self.update_edges(points=(t_current.k_values, t_next.k_values), tile_id=t.tile_id)
                else:
                    self.update_edges(points=(t_next.k_values, t_current.k_values), tile_id=t.tile_id)

            bt_match = False
            for bt in self.base_tiles:
                if bt.compare_angles(t):
                    t.tile_group = bt.tile_group
                    bt_match = True
                    break

            if not bt_match:
                t.tile_group = len(self.base_tiles)
                self.base_tiles.append(t)

            self.tiles[t.tile_id] = t

    def remove_unconnected_tiles(self):

        # Could add neighbor ids into the tile class?, that would remove the edge class, then add exception into the tile class?
        neighbors = defaultdict(list)
        for edge in self.edges.values():
            if len(edge.tile_ids) == 2:
                neighbors[edge.tile_ids[0]].append(edge.tile_ids[1])
                neighbors[edge.tile_ids[1]].append(edge.tile_ids[0])
            elif len(edge.tile_ids) > 2:
                print('Error')

        # Find center-ish tile
        center_tile = self.tiles[list(self.tiles.keys())[0]]
        center_point = center_tile.tile_points[0]
        center_distance = center_point.location[0]**2 + center_point.location[1]**2
        for tile in self.tiles.values():
            tile_point = tile.tile_points[0]
            tile_distance = tile_point.location[0]**2 + tile_point.location[1]**2
            if tile_distance < center_distance:
                center_tile = tile
                center_point = tile_point
                center_distance = tile_distance

        center_tile.connected = True
        connected_tiles = deque([center_tile])

        while connected_tiles:
            current_tile = connected_tiles.pop()
            for neighbor in neighbors[current_tile.tile_id]:
                neighbor_tile = self.tiles[neighbor]
                if not neighbor_tile.connected:
                    neighbor_tile.connected = True
                    connected_tiles.append(neighbor_tile)

        # Now, delete the not connected tiles
        for k in list(self.tiles.keys()):
            if not self.tiles[k].connected:
                del self.tiles[k]

        remove_edges = []
        for edge in self.edges.values():
            for t in edge.tile_ids.copy():
                if t not in self.tiles:
                    edge.tile_ids.remove(t)

            if not edge.tile_ids:
                remove_edges.append(edge.points)

        for edge in remove_edges:
            del self.edges[edge]

    def draw_image(self, image_file):
        closest_distance = np.inf

        for edge in self.edges.values():
            if len(edge.tile_ids) == 1:
                for p in edge.points:
                    p_distance = self.points[p].location[0] ** 2 + self.points[p].location[1] ** 2
                    if p_distance < closest_distance:
                        closest_distance = p_distance

        closest_distance = np.sqrt(closest_distance)

        image_bounds = closest_distance / np.sqrt(2)

        if not self.colors:
            for i in range(len(self.base_tiles) + 1):
                self.colors.append(self.random_color())

        fig, ax = plt.subplots(figsize=(5, 5))

        polygons = []
        polygon_colors = []
        for t in self.tiles.values():
            locations = []
            for p in t.tile_points:
                locations.append(p.location)

            polygons.append(locations)
            polygon_colors.append(self.colors[t.tile_group])
                
            # p = Polygon(locations, facecolor = self.colors[t.tile_group], edgecolor=self.colors[-1])
            # ax.add_patch(p)
            
        polygon_collection = PolyCollection(polygons, facecolor=polygon_colors, edgecolor=self.colors[-1])
        ax.add_collection(polygon_collection)
        ax.set_aspect('equal')
        plt.axis([-image_bounds, image_bounds, -image_bounds, image_bounds])
        plt.axis('off')
        plt.tight_layout(pad=0)

        plt.savefig(image_file, dpi=150)
        plt.close()

    def random_color(self):
        converts_rgb = False

        a = self.rng.uniform(0, 1)

        while not converts_rgb:
            L = self.rng.uniform(0, 1)
            a = self.rng.uniform(-.3, .3)
            b = self.rng.uniform(-.3, .3)

            rgb = colour.XYZ_to_sRGB(colour.Oklab_to_XYZ([L, a, b]))
            if ((0 <= rgb[0] and rgb[0] <= 1) and
                (0 <= rgb[1] and rgb[1] <= 1) and
                (0 <= rgb[2] and rgb[2] <= 1)):
                converts_rgb = True

        return rgb

In [112]:
mg1 = MultiGrid(seed=23)
mg1.create_tiles()
mg1.remove_unconnected_tiles()
mg1.draw_image(f'output/graph.png')

In [108]:
for i in range(10):
    mg = MultiGrid(seed=i, colors=[])
    mg.create_tiles()
    mg.remove_unconnected_tiles()
    mg.draw_image(f'output/tests/graph_{i:02d}.png')
    print(mg.seed, mg.grid_count, mg.grid_bounds)

0 17 [-7, 8]
1 11 [-6, 7]
2 17 [-4, 5]
3 16 [-2, 3]
4 15 [-9, 10]
5 14 [-8, 9]
6 10 [-6, 7]
7 19 [-7, 8]
8 15 [-4, 5]
9 10 [-8, 9]


In [ ]:
# Could add neighbor ids into the tile class?, that would remove the edge class, then add exception into the tile class?
time it to see if points can be removed
make sure I can get the common tiling patterns

In [100]:
l = [1, 2, 3, 4, 5]

for l0, l1 in zip(l, l[1:] + [l[0]]):
    print(l0, l1)

1 2
2 3
3 4
4 5
5 1


In [99]:
l[1:] + [l[0]]

[2, 3, 4, 5, 1]